In [1]:


import pygame
import sys
import tkinter as tk
from tkinter import messagebox
import math

pygame.init()

WIDTH, HEIGHT = 600, 600

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 180, 0)
ORANGE = (255, 140, 0)
VIOLET = (138, 43, 226)
SKY_BLUE = (135, 206, 235)

X_MIN, X_MAX = -50, 50
Y_MIN, Y_MAX = -50, 50

font_name = "Arial"
font_size = 14

def logical_to_screen(x, y):
    screen_x = int((x - X_MIN) / (X_MAX - X_MIN) * WIDTH)
    screen_y = int(HEIGHT - ((y - Y_MIN) / (Y_MAX - Y_MIN) * HEIGHT))
    return screen_x, screen_y

def draw_axes(screen, font):
    pygame.draw.line(screen, BLACK, logical_to_screen(X_MIN, 0), logical_to_screen(X_MAX, 0), 2)
    pygame.draw.line(screen, BLACK, logical_to_screen(0, Y_MIN), logical_to_screen(0, Y_MAX), 2)

    for x in range(X_MIN, X_MAX + 1, 10):
        sx, sy = logical_to_screen(x, 0)
        pygame.draw.line(screen, BLACK, (sx, sy - 5), (sx, sy + 5), 1)
        label = font.render(str(x), True, BLACK)
        screen.blit(label, (sx - 10, sy + 10))

    for y in range(Y_MIN, Y_MAX + 1, 10):
        sx, sy = logical_to_screen(0, y)
        pygame.draw.line(screen, BLACK, (sx - 5, sy), (sx + 5, sy), 1)
        if y != 0:
            label = font.render(str(y), True, BLACK)
            screen.blit(label, (sx - 30, sy - 7))

def draw_dotted_circle(screen, center, radius, color, line_width=2, dash_length=5):
    circumference = 2 * math.pi * radius
    num_dashes = int(circumference / (dash_length * 2))
    for i in range(num_dashes):
        start_angle = (i * 2 * math.pi) / num_dashes
        end_angle = ((i + 0.5) * 2 * math.pi) / num_dashes
        steps = max(3, int(abs(end_angle - start_angle) * radius / 2))
        points = []
        for j in range(steps + 1):
            angle = start_angle + (end_angle - start_angle) * j / steps
            x = center[0] + radius * math.cos(angle)
            y = center[1] + radius * math.sin(angle)
            points.append((int(x), int(y)))
        if len(points) > 1:
            pygame.draw.lines(screen, color, False, points, line_width)

def draw_via(screen, center_x, diameters, font, backdrill_enabled=False, backdrill_diameter=0, backdrill_antipad=0):
    colors = [RED, GREEN, BLUE]
    for d, color in zip(diameters, colors):
        radius = d / 2
        cx, cy = logical_to_screen(center_x, 0)
        px_radius = int(radius / (X_MAX - X_MIN) * WIDTH)
        pygame.draw.circle(screen, color, (cx, cy), px_radius, 2)
    if backdrill_enabled and backdrill_diameter > 0:
        backdrill_radius = backdrill_diameter / 2
        px_backdrill_radius = int(backdrill_radius / (X_MAX - X_MIN) * WIDTH)
        draw_dotted_circle(screen, (cx, cy), px_backdrill_radius, VIOLET, 2)
        if backdrill_antipad > 0:
            antipad_radius = backdrill_antipad / 2
            px_antipad_radius = int(antipad_radius / (X_MAX - X_MIN) * WIDTH)
            draw_dotted_circle(screen, (cx, cy), px_antipad_radius, SKY_BLUE, 2)

def draw_trace(screen, center_x, width, height, color):
    left = center_x - width / 2
    bottom = -height / 2
    screen_left, screen_top = logical_to_screen(left, bottom + height)
    screen_width = int(width / (X_MAX - X_MIN) * WIDTH)
    screen_height = int(height / (Y_MAX - Y_MIN) * HEIGHT)
    rect = pygame.Rect(screen_left, screen_top, screen_width, screen_height)
    pygame.draw.rect(screen, color, rect, 2)
    return rect

class ViaTraceVisualizer:
    def __init__(self):
        self.via1_diameters = [6.9, 14.9, 24.9]
        self.via2_diameters = [6.9, 14.9, 24.9]
        self.d2m = 9
        self.bd2m = 5
        self.via_pitch = 39.37

        self.via1_backdrill_enabled = False
        self.via2_backdrill_enabled = False
        self.via1_backdrill = 14.9
        self.via2_backdrill = 14.9

        self.via1_backdrill_antipad = 0
        self.via2_backdrill_antipad = 0

        self.trace1_width = 4.5
        self.trace2_width = 4.5
        self.trace_spacing = 5
        self.trace_height = 40

        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Vias and Traces Visualization with Backdrill")

        self.font = pygame.font.SysFont(font_name, font_size)
        self.font_bold = pygame.font.SysFont(font_name, font_size, bold=True)

        self.root = tk.Tk()
        self.root.title("Input Via & Trace Parameters")
        self.create_widgets()

        self.running = True
        self.clock = pygame.time.Clock()

    def create_widgets(self):
        tk.Label(self.root, text="All units are in mils", font=("Arial", 12, "bold")).grid(row=0, column=0, columnspan=8, pady=5)
        tk.Label(self.root, text="Drill").grid(row=1, column=1, padx=5, pady=2)
        tk.Label(self.root, text="Pad").grid(row=1, column=2, padx=5, pady=2)
        tk.Label(self.root, text="Antipad").grid(row=1, column=3, padx=5, pady=2)
        tk.Label(self.root, text="Backdrill").grid(row=1, column=5, padx=5, pady=2)
        tk.Label(self.root, text="BD_Antipad").grid(row=1, column=7, padx=5, pady=2)

        tk.Label(self.root, text="Via 1:").grid(row=2, column=0, padx=5, pady=5, sticky="e")
        self.via1_entries = []
        drill1_entry = tk.Entry(self.root, width=8)
        drill1_entry.grid(row=2, column=1, padx=5)
        drill1_entry.insert(0, str(self.via1_diameters[0]))
        drill1_entry.bind('<KeyRelease>', lambda e: self.calculate_values(1))
        self.via1_entries.append(drill1_entry)
        pad1_entry = tk.Entry(self.root, width=8)
        pad1_entry.grid(row=2, column=2, padx=5)
        pad1_entry.insert(0, str(self.via1_diameters[1]))
        self.via1_entries.append(pad1_entry)
        antipad1_entry = tk.Entry(self.root, width=8)
        antipad1_entry.grid(row=2, column=3, padx=5)
        antipad1_entry.insert(0, str(self.via1_diameters[2]))
        antipad1_entry.bind('<KeyRelease>', lambda e: self.validate_antipad(1))
        self.via1_entries.append(antipad1_entry)
        self.via1_backdrill_var = tk.BooleanVar()
        via1_backdrill_check = tk.Checkbutton(self.root, text="Enable BD",
                                              variable=self.via1_backdrill_var,
                                              command=lambda: self.toggle_backdrill(1))
        via1_backdrill_check.grid(row=2, column=4, padx=5, pady=5, sticky="w")
        self.via1_backdrill_entry = tk.Entry(self.root, width=8, state='disabled')
        self.via1_backdrill_entry.grid(row=2, column=5, padx=5)
        self.via1_backdrill_entry.insert(0, str(self.via1_backdrill))
        self.via1_backdrill_entry.bind('<KeyRelease>', lambda e: self.validate_backdrill(1))
        self.via1_backdrill_antipad_entry = tk.Entry(self.root, width=8, state='disabled')
        self.via1_backdrill_antipad_entry.grid(row=2, column=7, padx=5)
        self.via1_backdrill_antipad_entry.bind('<KeyRelease>', lambda e: self.validate_backdrill_antipad(1))

        tk.Label(self.root, text="Via 2:").grid(row=3, column=0, padx=5, pady=5, sticky="e")
        self.via2_entries = []
        drill2_entry = tk.Entry(self.root, width=8)
        drill2_entry.grid(row=3, column=1, padx=5)
        drill2_entry.insert(0, str(self.via2_diameters[0]))
        drill2_entry.bind('<KeyRelease>', lambda e: self.calculate_values(2))
        self.via2_entries.append(drill2_entry)
        pad2_entry = tk.Entry(self.root, width=8)
        pad2_entry.grid(row=3, column=2, padx=5)
        pad2_entry.insert(0, str(self.via2_diameters[1]))
        self.via2_entries.append(pad2_entry)
        antipad2_entry = tk.Entry(self.root, width=8)
        antipad2_entry.grid(row=3, column=3, padx=5)
        antipad2_entry.insert(0, str(self.via2_diameters[2]))
        antipad2_entry.bind('<KeyRelease>', lambda e: self.validate_antipad(2))
        self.via2_entries.append(antipad2_entry)
        self.via2_backdrill_var = tk.BooleanVar()
        via2_backdrill_check = tk.Checkbutton(self.root, text="Enable BD",
                                              variable=self.via2_backdrill_var,
                                              command=lambda: self.toggle_backdrill(2))
        via2_backdrill_check.grid(row=3, column=4, padx=5, pady=5, sticky="w")
        self.via2_backdrill_entry = tk.Entry(self.root, width=8, state='disabled')
        self.via2_backdrill_entry.grid(row=3, column=5, padx=5)
        self.via2_backdrill_entry.insert(0, str(self.via2_backdrill))
        self.via2_backdrill_entry.bind('<KeyRelease>', lambda e: self.validate_backdrill(2))
        self.via2_backdrill_antipad_entry = tk.Entry(self.root, width=8, state='disabled')
        self.via2_backdrill_antipad_entry.grid(row=3, column=7, padx=5)
        self.via2_backdrill_antipad_entry.bind('<KeyRelease>', lambda e: self.validate_backdrill_antipad(2))

       

     # Assuming the last main input row is row=4, so we start from row=5 or 6 for d2m and BD2M

# Row 6: d2m
        tk.Label(self.root, text="d2m:").grid(row=6, column=4, padx=5, pady=5, sticky="e")
        self.d2m_entry = tk.Entry(self.root, width=8)
        self.d2m_entry.grid(row=6, column=5, padx=5)
        self.d2m_entry.insert(0, str(self.d2m))
        self.d2m_entry.bind('<KeyRelease>', lambda e: self.update_antipads())

        # Row 7: BD2M
        tk.Label(self.root, text="BD2M:").grid(row=7, column=4, padx=5, pady=5, sticky="e")
        self.bd2m_entry = tk.Entry(self.root, width=8, state='disabled')
        self.bd2m_entry.grid(row=7, column=5, padx=5)
        self.bd2m_entry.insert(0, str(self.bd2m))
        self.bd2m_entry.bind('<KeyRelease>', lambda e: self.update_backdrill_antipads())


        tk.Label(self.root, text="Via Pitch:").grid(row=5, column=0, padx=5, pady=5, sticky="e")
        self.via_pitch_entry = tk.Entry(self.root, width=8)
        self.via_pitch_entry.grid(row=5, column=1, padx=5)
        self.via_pitch_entry.insert(0, str(self.via_pitch))
        tk.Label(self.root, text="Trace 1 Width:").grid(row=6, column=0, padx=5, pady=5, sticky="e")
        self.trace1_width_entry = tk.Entry(self.root, width=8)
        self.trace1_width_entry.grid(row=6, column=1, padx=5)
        self.trace1_width_entry.insert(0, str(self.trace1_width))
        tk.Label(self.root, text="Trace 2 Width:").grid(row=6, column=2, padx=5, pady=5, sticky="e")
        self.trace2_width_entry = tk.Entry(self.root, width=8)
        self.trace2_width_entry.grid(row=6, column=3, padx=5)
        self.trace2_width_entry.insert(0, str(self.trace2_width))
        tk.Label(self.root, text="Trace to Trace Spacing (SP):").grid(row=7, column=0, padx=5, pady=5, sticky="e")
        self.trace_spacing_entry = tk.Entry(self.root, width=8)
        self.trace_spacing_entry.grid(row=7, column=1, padx=5)
        self.trace_spacing_entry.insert(0, str(self.trace_spacing))
        # tk.Label(self.root, text="Trace Height:").grid(row=8, column=0, padx=5, pady=5, sticky="e")
        # self.trace_height_entry = tk.Entry(self.root, width=8)
        # self.trace_height_entry.grid(row=8, column=1, padx=5)
        # self.trace_height_entry.insert(0, str(self.trace_height))
        draw_button = tk.Button(self.root, text="Draw", command=self.update_and_draw)
        draw_button.grid(row=9, column=0, pady=10)
        clear_button = tk.Button(self.root, text="Clear", command=self.clear_all)
        clear_button.grid(row=9, column=1, pady=10)

    def toggle_backdrill(self, via_num):
        if via_num == 1:
            self.via1_backdrill_enabled = self.via1_backdrill_var.get()
            entries = [self.via1_backdrill_entry, self.via1_backdrill_antipad_entry]
        else:
            self.via2_backdrill_enabled = self.via2_backdrill_var.get()
            entries = [self.via2_backdrill_entry, self.via2_backdrill_antipad_entry]
        if (self.via1_backdrill_enabled or self.via2_backdrill_enabled):
            self.bd2m_entry.config(state='normal')
            if not self.bd2m_entry.get():
                self.bd2m_entry.insert(0, "5")
        else:
            self.bd2m_entry.config(state='disabled')
        if (via_num == 1 and self.via1_backdrill_enabled) or (via_num == 2 and self.via2_backdrill_enabled):
            for entry in entries:
                entry.config(state='normal')
            self.calculate_backdrill_defaults()
            self.calculate_backdrill_antipad(via_num)
        else:
            for entry in entries:
                entry.config(state='disabled')
        self.update_and_draw()

    def calculate_backdrill_defaults(self):
        try:
            drill1 = float(self.via1_entries[0].get()) if self.via1_entries[0].get() else 6.9
            drill2 = float(self.via2_entries[0].get()) if self.via2_entries[0].get() else 6.9
            default_backdrill1 = drill1 + 8
            default_backdrill2 = drill2 + 8
            self.via1_backdrill_entry.delete(0, tk.END)
            self.via1_backdrill_entry.insert(0, str(default_backdrill1))
            self.via2_backdrill_entry.delete(0, tk.END)
            self.via2_backdrill_entry.insert(0, str(default_backdrill2))
        except ValueError:
            pass

    def calculate_values(self, via_num):
        self.update_antipads()
        if self.via1_backdrill_enabled or self.via2_backdrill_enabled:
            self.calculate_backdrill_defaults()
            self.update_backdrill_antipads()

    def update_antipads(self):
        try:
            d2m = float(self.d2m_entry.get()) if self.d2m_entry.get() else 9
            for idx, entries in enumerate([self.via1_entries, self.via2_entries], start=1):
                drill = float(entries[0].get()) if entries[0].get() else 0
                calculated_antipad = drill + 2 * d2m
                entries[2].delete(0, tk.END)
                entries[2].insert(0, str(calculated_antipad))
        except ValueError:
            pass

    def update_backdrill_antipads(self):
        try:
            bd2m = float(self.bd2m_entry.get()) if self.bd2m_entry.get() else 5
            for via_num, (backdrill_entry, antipad_entry) in enumerate(
                [(self.via1_backdrill_entry, self.via1_backdrill_antipad_entry),
                 (self.via2_backdrill_entry, self.via2_backdrill_antipad_entry)], start=1):
                backdrill = float(backdrill_entry.get()) if backdrill_entry.get() else 0
                calculated_antipad = backdrill + 2 * bd2m
                antipad_entry.delete(0, tk.END)
                antipad_entry.insert(0, str(calculated_antipad))
                if via_num == 1:
                    self.via1_backdrill_antipad = calculated_antipad
                else:
                    self.via2_backdrill_antipad = calculated_antipad
        except ValueError:
            pass

    def calculate_backdrill_antipad(self, via_num):
        try:
            bd2m = float(self.bd2m_entry.get()) if self.bd2m_entry.get() else 5
            if via_num == 1:
                backdrill = float(self.via1_backdrill_entry.get()) if self.via1_backdrill_entry.get() else 0
                calculated_antipad = backdrill + 2 * bd2m
                self.via1_backdrill_antipad_entry.delete(0, tk.END)
                self.via1_backdrill_antipad_entry.insert(0, str(calculated_antipad))
                self.via1_backdrill_antipad = calculated_antipad
            elif via_num == 2:
                backdrill = float(self.via2_backdrill_entry.get()) if self.via2_backdrill_entry.get() else 0
                calculated_antipad = backdrill + 2 * bd2m
                self.via2_backdrill_antipad_entry.delete(0, tk.END)
                self.via2_backdrill_antipad_entry.insert(0, str(calculated_antipad))
                self.via2_backdrill_antipad = calculated_antipad
        except ValueError:
            pass

    def validate_antipad(self, via_num):
        pass

    def validate_backdrill(self, via_num):
        pass

    def validate_backdrill_antipad(self, via_num):
        pass

    def clear_all(self):
        self.via1_diameters = [6.9, 14.9, 24.9]
        self.via2_diameters = [6.9, 14.9, 24.9]
        self.d2m = 9
        self.bd2m = 5
        self.via_pitch = 39.37
        self.via1_backdrill_enabled = False
        self.via2_backdrill_enabled = False
        self.via1_backdrill = 14.9
        self.via2_backdrill = 14.9
        self.trace1_width = 4.5
        self.trace2_width = 4.5
        self.trace_spacing = 5
        self.trace_height = 40

        self.via1_backdrill_var.set(False)
        self.via2_backdrill_var.set(False)

        for i in range(3):
            self.via1_entries[i].delete(0, tk.END)
            self.via1_entries[i].insert(0, str(self.via1_diameters[i]))
            self.via2_entries[i].delete(0, tk.END)
            self.via2_entries[i].insert(0, str(self.via2_diameters[i]))

        self.d2m_entry.delete(0, tk.END)
        self.d2m_entry.insert(0, "9")
        self.bd2m_entry.config(state='normal')
        self.bd2m_entry.delete(0, tk.END)
        self.bd2m_entry.insert(0, "5")
        if not (self.via1_backdrill_var.get() or self.via2_backdrill_var.get()):
            self.bd2m_entry.config(state='disabled')

        self.via1_backdrill_entry.config(state='normal')
        self.via1_backdrill_entry.delete(0, tk.END)
        self.via1_backdrill_entry.insert(0, str(self.via1_backdrill))
        self.via1_backdrill_entry.config(state='disabled')

        self.via2_backdrill_entry.config(state='normal')
        self.via2_backdrill_entry.delete(0, tk.END)
        self.via2_backdrill_entry.insert(0, str(self.via2_backdrill))
        self.via2_backdrill_entry.config(state='disabled')

        self.via1_backdrill_antipad_entry.config(state='normal')
        self.via1_backdrill_antipad_entry.delete(0, tk.END)
        self.via1_backdrill_antipad_entry.insert(0, "0")
        self.via1_backdrill_antipad_entry.config(state='disabled')

        self.via2_backdrill_antipad_entry.config(state='normal')
        self.via2_backdrill_antipad_entry.delete(0, tk.END)
        self.via2_backdrill_antipad_entry.insert(0, "0")
        self.via2_backdrill_antipad_entry.config(state='disabled')

        self.via_pitch_entry.delete(0, tk.END)
        self.via_pitch_entry.insert(0, str(self.via_pitch))

        self.trace1_width_entry.delete(0, tk.END)
        self.trace1_width_entry.insert(0, str(self.trace1_width))

        self.trace2_width_entry.delete(0, tk.END)
        self.trace2_width_entry.insert(0, str(self.trace2_width))

        self.trace_spacing_entry.delete(0, tk.END)
        self.trace_spacing_entry.insert(0, str(self.trace_spacing))

        # self.trace_height_entry.delete(0, tk.END)
        # self.trace_height_entry.insert(0, str(self.trace_height))

        self.draw()

    def update_and_draw(self):
        try:
            self.via1_diameters = [float(e.get()) for e in self.via1_entries]
            self.via2_diameters = [float(e.get()) for e in self.via2_entries]
            self.d2m = float(self.d2m_entry.get())
            self.bd2m = float(self.bd2m_entry.get()) if self.bd2m_entry['state'] == 'normal' else 5
            self.via_pitch = float(self.via_pitch_entry.get())

            errors = []
            d2m = self.d2m
            for idx, entries in enumerate([self.via1_entries, self.via2_entries], start=1):
                drill = float(entries[0].get()) if entries[0].get() else 0
                antipad = float(entries[2].get()) if entries[2].get() else 0
                min_antipad = drill + 2 * d2m
                if antipad < min_antipad and min_antipad > 0:
                    errors.append(f"Via {idx} Antipad ({antipad}) cannot be less than Drill + 2×d2m ({min_antipad:.3f})")
            bd2m = self.bd2m
            if self.via1_backdrill_enabled:
                self.via1_backdrill = float(self.via1_backdrill_entry.get())
                drill = float(self.via1_entries[0].get()) if self.via1_entries[0].get() else 0
                backdrill = float(self.via1_backdrill_entry.get()) if self.via1_backdrill_entry.get() else 0
                if backdrill < drill and drill > 0:
                    errors.append(f"Via 1 Backdrill ({backdrill}) cannot be less than Drill diameter ({drill})")
                antipad = float(self.via1_backdrill_antipad_entry.get()) if self.via1_backdrill_antipad_entry.get() else 0
                min_antipad = backdrill + 2 * bd2m
                if antipad < min_antipad and min_antipad > 0:
                    errors.append(f"Via 1 Backdrill Antipad ({antipad}) cannot be less than Backdrill + 2×BD2M ({min_antipad:.3f})")
                self.via1_backdrill_antipad = antipad
            if self.via2_backdrill_enabled:
                self.via2_backdrill = float(self.via2_backdrill_entry.get())
                drill = float(self.via2_entries[0].get()) if self.via2_entries[0].get() else 0
                backdrill = float(self.via2_backdrill_entry.get()) if self.via2_backdrill_entry.get() else 0
                if backdrill < drill and drill > 0:
                    errors.append(f"Via 2 Backdrill ({backdrill}) cannot be less than Drill diameter ({drill})")
                antipad = float(self.via2_backdrill_antipad_entry.get()) if self.via2_backdrill_antipad_entry.get() else 0
                min_antipad = backdrill + 2 * bd2m
                if antipad < min_antipad and min_antipad > 0:
                    errors.append(f"Via 2 Backdrill Antipad ({antipad}) cannot be less than Backdrill + 2×BD2M ({min_antipad:.3f})")
                self.via2_backdrill_antipad = antipad

            self.trace1_width = float(self.trace1_width_entry.get())
            self.trace2_width = float(self.trace2_width_entry.get())
            self.trace_spacing = float(self.trace_spacing_entry.get())
            self.trace_height = 40

            if self.via_pitch > 60:
                self.via_pitch = 60
            if self.trace_spacing < 0:
                self.trace_spacing = 0

            if errors:
                messagebox.showerror("Validation Error", "\n".join(errors))
                return

            self.draw()
        except ValueError:
            messagebox.showerror("Invalid input", "Please enter valid numeric values.")

   

    def draw(self):
        self.screen.fill(WHITE)
        margin = 18  # margin from axis for text
        units_text = self.font_bold.render("All units are in mils- screen", True, BLACK)
        self.screen.blit(units_text, (margin, margin // 2))

        draw_axes(self.screen, self.font)

        via_x1 = -self.via_pitch / 2
        via_x2 = self.via_pitch / 2
        via_c1 = logical_to_screen(via_x1, 0)
        via_c2 = logical_to_screen(via_x2, 0)
        pygame.draw.line(self.screen, GREEN, via_c1, via_c2, 3)
       
        draw_via(self.screen, via_x1, self.via1_diameters, self.font,
                 self.via1_backdrill_enabled, self.via1_backdrill if self.via1_backdrill_enabled else 0,
                 self.via1_backdrill_antipad if self.via1_backdrill_enabled else 0)
        draw_via(self.screen, via_x2, self.via2_diameters, self.font,
                 self.via2_backdrill_enabled, self.via2_backdrill if self.via2_backdrill_enabled else 0,
                 self.via2_backdrill_antipad if self.via2_backdrill_enabled else 0)

        # --- Trace positions and gap calculations ---
        show_gaps = False
        single_trace = False
        trace_left = trace_right = None

        if self.trace1_width > 0 and self.trace2_width > 0:
            # Two traces
            total_gap = self.trace_spacing + self.trace1_width / 2 + self.trace2_width / 2
            trace_x1 = -total_gap / 2
            trace_x2 = total_gap / 2
            rect1 = draw_trace(self.screen, trace_x1, self.trace1_width, self.trace_height, ORANGE)
            rect2 = draw_trace(self.screen, trace_x2, self.trace2_width, self.trace_height, ORANGE)
            pygame.draw.line(
                self.screen, BLUE,
                (rect1.right, rect1.top + rect1.height // 2),
                (rect2.left, rect2.top + rect2.height // 2),
                2
            )
            trace1_left = trace_x1 - (self.trace1_width / 2)
            trace2_right = trace_x2 + (self.trace2_width / 2)
            show_gaps = True
            trace_left = trace1_left
            trace_right = trace2_right
        elif self.trace1_width > 0 and self.trace2_width <= 0:
            # Only trace1, centered
            if self.via1_backdrill_enabled and self.via2_backdrill_enabled:
                left_edge = via_x1 + (self.via1_backdrill_antipad / 2)
                right_edge = via_x2 - (self.via2_backdrill_antipad / 2)
            else:
                left_edge = via_x1 + (self.via1_diameters[2] / 2)
                right_edge = via_x2 - (self.via2_diameters[2] / 2)
            trace_x1 = (left_edge + right_edge) / 2
            rect1 = draw_trace(self.screen, trace_x1, self.trace1_width, self.trace_height, ORANGE)
            trace_left = trace_x1 - (self.trace1_width / 2)
            trace_right = trace_x1 + (self.trace1_width / 2)
            show_gaps = True
            single_trace = True
        elif self.trace2_width > 0 and self.trace1_width <= 0:
            # Only trace2, centered
            if self.via1_backdrill_enabled and self.via2_backdrill_enabled:
                left_edge = via_x1 + (self.via1_backdrill_antipad / 2)
                right_edge = via_x2 - (self.via2_backdrill_antipad / 2)
            else:
                left_edge = via_x1 + (self.via1_diameters[2] / 2)
                right_edge = via_x2 - (self.via2_diameters[2] / 2)
            trace_x2 = (left_edge + right_edge) / 2
            rect2 = draw_trace(self.screen, trace_x2, self.trace2_width, self.trace_height, ORANGE)
            trace_left = trace_x2 - (self.trace2_width / 2)
            trace_right = trace_x2 + (self.trace2_width / 2)
            show_gaps = True
            single_trace = True
        else:
            # Both traces are zero, do not show gaps
            show_gaps = False

        # --- Calculate and display gaps ---
        y_antipad = HEIGHT - margin * 3
        y_bd = HEIGHT - margin * 2

        if show_gaps:
            via1_antipad_edge = via_x1 + (self.via1_diameters[2] / 2)
            via2_antipad_edge = via_x2 - (self.via2_diameters[2] / 2)
            via1_bdantipad_edge = via_x1 + (self.via1_backdrill_antipad / 2)
            via2_bdantipad_edge = via_x2 - (self.via2_backdrill_antipad / 2)

            antipad_LGap = trace_left - via1_antipad_edge
            antipad_RGap = via2_antipad_edge - trace_right
            bdantipad_LGap = trace_left - via1_bdantipad_edge
            bdantipad_RGap = via2_bdantipad_edge - trace_right

            lgap_text = self.font_bold.render(f"antipad_LGap: {antipad_LGap:.2f}", True, BLACK)
            rgap_text = self.font_bold.render(f"antipad_RGap: {antipad_RGap:.2f}", True, BLACK)
            self.screen.blit(lgap_text, (margin, y_antipad))
            text_width = rgap_text.get_width()
            self.screen.blit(rgap_text, (WIDTH - text_width - margin, y_antipad))
            bd_lgap_text = self.font_bold.render(f"BDantipad_Lgap: {bdantipad_LGap:.2f}", True, BLACK)
            bd_rgap_text = self.font_bold.render(f"BDantipad_Rgap: {bdantipad_RGap:.2f}", True, BLACK)
            self.screen.blit(bd_lgap_text, (margin, y_bd))
            text_width = bd_rgap_text.get_width()
            self.screen.blit(bd_rgap_text, (WIDTH - text_width - margin, y_bd))

        # --- Routing channel values (always show before, after only if BD enabled) ---
        y_dist = HEIGHT - margin * 6
        via1_outer = via_x1 + (self.via1_diameters[2] / 2)
        via2_outer = via_x2 - (self.via2_diameters[2] / 2)
        antipad_to_antipad = via2_outer - via1_outer
        dist_text = self.font_bold.render(
            f"Available Routing Channel before BackDrill - {antipad_to_antipad:.2f}", True, BLACK)
        self.screen.blit(dist_text, (margin, y_dist))

        if self.via1_backdrill_enabled or self.via2_backdrill_enabled:
            def get_max_radius(diameters, bd_enabled, bd, bd_antipad):
                circles = [diameters[0], diameters[1], diameters[2]]
                if bd_enabled:
                    circles.append(bd)
                    circles.append(bd_antipad)
                return max(circles) / 2

            via1_max = get_max_radius(self.via1_diameters, self.via1_backdrill_enabled, self.via1_backdrill, self.via1_backdrill_antipad)
            via2_max = get_max_radius(self.via2_diameters, self.via2_backdrill_enabled, self.via2_backdrill, self.via2_backdrill_antipad)
            via1_bd = via_x1 + via1_max
            via2_bd = via_x2 - via2_max
            big_dist = via2_bd - via1_bd
            offset = dist_text.get_height() + 5
            dist_text2 = self.font_bold.render(
                f"Available Routing Channel after BackDrill - {big_dist:.2f}", True, BLACK)
            self.screen.blit(dist_text2, (margin, y_dist + offset))

        # --- Legend (top-right) ---
        legend_items = [
            ("Drill", RED, "circle", False),
            ("Pad", GREEN, "circle", False),
            ("Antipad", BLUE, "circle", False),
            ("Backdrill", VIOLET, "circle", True),
            ("BD_Antipad", SKY_BLUE, "circle", True),
            ("Trace", ORANGE, "rect", False),
        ]
        legend_x = WIDTH - 140
        legend_y = 20
        spacing = 28
        radius = 10
        rect_w = 22
        rect_h = 10

        for i, (label, color, shape, dotted) in enumerate(legend_items):
            y = legend_y + i * spacing
            if shape == "circle":
                center = (legend_x, y + radius)
                if dotted:
                    draw_dotted_circle(self.screen, center, radius, color, 3, dash_length=4)
                else:
                    pygame.draw.circle(self.screen, color, center, radius, 0)
            elif shape == "rect":
                rect = pygame.Rect(legend_x - rect_w // 2, y + radius - rect_h // 2, rect_w, rect_h)
                pygame.draw.rect(self.screen, color, rect, 0)
            text = self.font.render(label, True, BLACK)
            self.screen.blit(text, (legend_x + 30, y + radius - 8))
        pygame.display.flip()


    def run(self):
        self.draw()
        while self.running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
            self.root.update()
            self.clock.tick(60)
        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    visualizer = ViaTraceVisualizer()
    visualizer.run()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\dell\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
